In [1]:
using PyPlot
using Random
using Distributions

In [ ]:
include("model.jl");
include("ensemble.jl");
include("diagnostics.jl");
include("optimize.jl");
include("plotting.jl");

In [ ]:
t = Array(2020:1.:2100);
β = 1.5 # damages (trillion $ / celsius^2)
utility_discount_rate = 0.014 # low value (stern review)
GWP = 100. # global world product (trillion $)
reduce_cost = 0.05*GWP
remove_cost = 0.15*GWP
geoeng_cost = 0.25*GWP
adapt_cost = 0.15*GWP
economics = Economics(
    β, utility_discount_rate,
    reduce_cost, remove_cost, geoeng_cost, adapt_cost
)

In [ ]:
Random.seed!(1)
d = Normal(0.65, 0.13)
feedback_samples = rand(d, 500)
feedback_samples[feedback_samples .>= 0.999] .= NaN
feedback_samples[feedback_samples .<= 0.] .= NaN

ECS₀ = 1.2
ECS_samples = ECS₀ ./ (1. .-feedback_samples);

In [ ]:
hist(ECS_samples, bins=0.:0.5:20.)
xlim([0,10])

In [ ]:
ensemble = Dict{String, ClimateModel}();
first = true
for member_id in 1:length(ECS_samples)
    ECS = ECS_samples[member_id]
    name = "member $member_id";
    controls = init_linear_controls(t);
    model = ClimateModel(name, ECS, t, controls, economics);
    optimize!(model)
    
    if CO₂(model)[end-1] < 1.e5
        ensemble[name] = model
    end
        
    if first; plot_setup(model); first=false; end
end

In [ ]:
plot_ensemble_stats(ensemble, t)